In [82]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import matplotlib

matplotlib.rcParams['figure.figsize'] = [12, 8]

import math
import numpy as np
import pickle

from interaction import Interaction
from environment import Environment
from DelightFish import Fish
from channel import Channel
from observer import Observer
from discriminator import Classifier
from fitness import Fitness
from optimizer import Optimizer

from turing_learning import test_simulation
from utils import generate_distortion, generate_fish, generate_replica_fish, generate_all_fish, run_simulation

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [92]:
conn_threshold = 100
run_time = 15
total_fish = 25
#num_fish = 20
#num_replica_fish = 1
k_ar = 0.003
max_speed = 9
arena_size = 300

In [93]:
def run_full_test(weights, real = False):
    arena_center = arena_size / 2.0
    initial_spread = 20
    fish_pos = initial_spread * np.random.rand(total_fish, 2) + arena_center - initial_spread / 2.0
    clock_freqs = 1
    verbose = False

    distortion = generate_distortion(type='none', n=arena_size)
    environment = Environment(
        node_pos=fish_pos,
        distortion=distortion,
        prob_type='binary',
        noise_magnitude=0,
        conn_thres=conn_threshold,
        verbose=verbose
    )
    interaction = Interaction(environment, verbose=verbose)
    channel = Channel(environment)
    
    # Have all real or all fake
    if real:
        n_fish = total_fish
        n_replica_fish = 0
    else:
        n_fish = 0
        n_replica_fish = total_fish
        
    fish = generate_all_fish(
        n_fish=n_fish,
        n_replica_fish= n_replica_fish,
        channel=channel, 
        interaction=interaction,
        k_coh = 0,
        k_ar = k_ar,
        alpha = 40,
        weights = weights,
        lim_neighbors=[0, math.inf],
        neighbor_weights=1.0,
        fish_max_speeds=max_speed,
        clock_freqs=clock_freqs,
        verbose=verbose
    )
    channel.set_nodes(fish)

    observer = Observer(fish=fish, environment=environment, channel=channel)
    fish_matrix = test_simulation(fish=fish, observer=observer, run_time=run_time)
    return fish_matrix


In [ ]:
# go through ten generations
opt = Optimizer()
opt.init_model(21, 50)
opt.init_classifier(46, 50)
for i in range(200):
    model_weights = opt.get_model_weights()
    classifier_weights = opt.get_classifier_weights()
    ideal_model = run_full_test(None, real = True)
    replica_models = [run_full_test(weights) for weights in model_weights]
    replica_models.insert(0, ideal_model)
    all_trials = np.stack(replica_models)
    total_classifiers = [Classifier(id = 1, weights = weights).classify_all_models(all_trials) for weights in classifier_weights]
    fitness_scorer = Fitness(total_classifiers)
    
    class_scores = fitness_scorer.score_classifiers()
    model_scores = fitness_scorer.score_models()
    print(class_scores)
    print(model_scores)
    
    opt.give_model_scores(model_scores)
    opt.give_classifier_scores(class_scores)
    

file = open('test1.obj', 'w')
pickle.dump((model_weights, classifier_weights), file)
file.close()

(25_w,50)-aCMA-ES (mu_w=14.0,w_1=14%) in dimension 21 (seed=721061, Sun Nov 18 16:42:59 2018)
(25_w,50)-aCMA-ES (mu_w=14.0,w_1=14%) in dimension 46 (seed=650845, Sun Nov 18 16:42:59 2018)
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait pa

[0.1284, 0.498, 0.4896, 0.5, 0.057600000000000005, 0.4092, 0.5, 0.7876000000000001, 0.05160000000000002, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.07560000000000003, 0.5076, 0.5108, 0.4636, 0.42519999999999997, 0.326, 0.5, 0.5, 0.46280000000000004, 0.4632, 0.5, 0.5, 0.4944, 0.5, 0.23879999999999998, 0.5, 0.5551999999999999, 0.5, 0.5, 0.0804, 0.466, 0.4676, 0.4996, 0.12680000000000002, 0.5, 0.5, 0.12960000000000002, 0.5288, 0.5, 0.5, 0.5, 0.5328, 0.1264, 0.5]
[0.4696, 0.5, 0.4384, 0.516, 0.5088, 0.512, 0.4552, 0.5, 0.4904, 0.5, 0.4808, 0.512, 0.5008, 0.4936, 0.4584, 0.5016, 0.5056, 0.516, 0.5, 0.4792, 0.4544, 0.4408, 0.4792, 0.4944, 0.5, 0.5016, 0.504, 0.5008, 0.4688, 0.5168, 0.5064, 0.3896, 0.4936, 0.5136, 0.5, 0.464, 0.4576, 0.5, 0.516, 0.5008, 0.5008, 0.4256, 0.52, 0.4992, 0.5, 0.504, 0.5176, 0.504, 0.5, 0.5016]
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait

Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
[0.23359999999999997, 0.5, 0.5, 0.4816, 0.5016, 0.5016, 0.5184, 0.368, 0.4732, 0.5004, 0.2552, 0.5, 0.4984, 0.5, 0.41559999999999997, 0.4648, 0.4716, 0.3916, 0.2748, 0.5004, 0.4844, 0.1592, 0.5, 0.3992, 0.21400000000000002, 0.5, 0.4272, 0.4132, 0.5004, 0.3216, 0.36119999999999997, 0.31

/Users/kbinney/thesis/UndercoverBot/replica_fish.py:182: RuntimeWarning: overflow encountered in exp
  event {Ping} -- The ping event instance


Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
[0.43679999999999997, 0.4272, 0.392, 0.5004, 0.6092, 0.4108, 0.44279999999999997, 0.5004, 0.4808, 0.4412, 0.5, 0.4672, 0.4852, 0.4756, 0.5, 0.5, 0.5012, 0.5, 0.5, 0.4996, 0.5, 0.4432, 0.5, 0.5, 0.5, 0.44279999999999997, 0.4532, 0.4376, 0.5, 0.426, 0.4444, 0.4792, 0.424, 0.5, 0.5, 0.4016, 0.5008, 0.5, 0.4424, 0.5, 0.4528, 0.5, 0.4616, 0.43720000000000003, 0.4816, 0.5012, 0.4064, 0.5, 0.5, 0.6224000000000001]
[0.4736, 0.4904, 0.6248, 0.8704, 0.46, 0.4704, 0.4664, 0.464, 0.4856, 0.8552, 0.46, 0

...

Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. 

Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. 

Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. 

Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
[0.48519999999999996, 0.7328, 0.5147999999999999, 0.5, 0.4812, 0.33640000000000003, 0.5347999999999999, 0.5, 0.2792, 0.5147999999999999, 0.456, 0.6688000000000001, 0.4948, 0.4968, 0.4552, 0.4772, 0.4624, 0.4624, 0.4744, 0.2972, 0.33199999999999996, 0.4152, 0.43520000000000003, 0.6808000000000001, 0.48200000000000004, 0.4756, 0.5224, 0.4576, 0.5008, 0.35600000000000004, 0.44680000000000003, 0.7136, 0.4328, 0.5008, 0.2524, 0.5384, 0.4812, 0.2772, 0.5052, 0.456, 0.482, 0.4796, 0.4948, 0.5, 0.47559999999999997, 0.4756, 0.4756, 0.39199999999999996, 0.33399999999999996, 0.4788]
[0.4144, 0.2864, 0.3728, 0.3832, 0.3352, 0.4224, 0.3744, 0.404, 0.4632, 0.3232, 0.2616, 0.3248, 0.3688, 0.3344, 0.3504, 0.408, 0.344, 0.4056, 0.2944, 0.3832, 0.3664, 0.3448, 0.3648, 0.2864, 0.3784, 0.4056, 0.3704, 0.4352, 0.3744, 0.4016, 0.3608, 0.3112,

Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. Thanks.
Please wait patiently 15 seconds. 

In [91]:
for model in replica_models:
    print(model.shape)
len(replica_models)

(25, 1, 2)
(25, 1, 2)
(25, 1, 2)
(25, 1, 2)
(25, 1, 2)
(25, 1, 2)
(25, 1, 2)
(25, 1, 2)
(25, 1, 2)
(25, 1, 2)
(25, 1, 2)
(25, 1, 2)
(25, 1, 2)
(25, 1, 2)
(25, 1, 2)
(25, 1, 2)
(25, 1, 2)
(25, 1, 2)
(25, 1, 2)
(25, 1, 2)
(25, 1, 2)
(25, 1, 2)
(25, 1, 2)
(25, 1, 2)
(25, 1, 2)
(25, 1, 2)
(25, 1, 2)
(25, 1, 2)
(25, 1, 2)
(25, 1, 2)
(25, 1, 2)
(25, 1, 2)
(25, 1, 2)
(25, 1, 2)
(25, 1, 2)
(25, 1, 2)
(25, 1, 2)
(25, 1, 2)
(25, 1, 2)
(25, 1, 2)
(25, 1, 2)
(25, 1, 2)
(25, 1, 2)
(25, 2, 2)
(25, 1, 2)
(25, 1, 2)
(25, 1, 2)
(25, 1, 2)
(25, 1, 2)
(25, 1, 2)
(25, 1, 2)


51

In [68]:
total_classifiers = [Classifier(id = 1, weights = [-1 for x in range(46)]).classify_all_models(all_trials) for weights in classifier_weights]
fitness_scorer = Fitness(total_classifiers)
class_scores = fitness_scorer.score_classifiers()
model_scores = fitness_scorer.score_models()
print(model_scores)


[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [61]:
total_classifiers = [Classifier(id = 1, weights = weights).classify_all_models(all_trials) for weights in classifier_weights]
fitness_scorer = Fitness(total_classifiers)

class_scores = fitness_scorer.score_classifiers()
model_scores = fitness_scorer.score_models()
print(class_scores)
print(model_scores)

[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [57]:
models = [data[1:, :] for data in total_classifiers]
print(len(models))
model_arr = np.stack(models, axis = 1)
print(model_arr.shape)
model_scores = np.mean(model_arr, axis = (1,2))
print(model_scores.shape)
model_scores.flatten().tolist()


50
(50, 50, 25)
(50,)


[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0]

In [59]:
weights = range(46)
classifier = Classifier(id = 1, weights = weights)
classifier.classify_models(fish_matrix)
all_classified = classifier.classify_all_models(all_trials)
total_classifiers = [all_classified for i in range(100)]

In [60]:
fitness_scorer = Fitness(total_classifiers)
print(len(fitness_scorer.score_classifiers()))
print(len(fitness_scorer.score_models()))

100
100


In [49]:
opt = Optimizer()

In [52]:
opt.init_model(17, 100)
opt.init_classifier(46, 100)

(50_w,100)-aCMA-ES (mu_w=27.0,w_1=8%) in dimension 17 (seed=967836, Mon Oct 29 12:56:29 2018)
(50_w,100)-aCMA-ES (mu_w=27.0,w_1=8%) in dimension 46 (seed=905871, Mon Oct 29 12:56:29 2018)


In [56]:
weights = opt.get_model_weights()[0]



In [31]:
all_return.shape


(100, 1, 10)

In [63]:
all_fish.shape



(100, 10, 5, 2)

In [73]:
weights.shape


AttributeError: 'range' object has no attribute 'shape'

In [80]:
classifier_weights

[array([4.61997272, 0.848501  , 0.67521526, 0.45558463, 6.16997087,
        0.08716378, 4.80000271, 6.25027001, 2.73772635, 6.75024821,
        1.66818858, 3.75046059, 1.26760439, 0.11349848, 2.51024299,
        2.96260077, 1.644022  , 3.73797226, 6.59720813, 0.03338364,
        0.31439095, 3.11157112, 4.6781033 , 0.97894509, 1.09439867,
        1.25700191, 4.17351992, 0.89799091, 3.8716744 , 3.71180871,
        3.79426995, 6.18974168, 3.05245208, 1.12172232, 6.46819621,
        1.45626932, 1.50035177, 2.83421906, 1.24749299, 0.36263485,
        3.44372168, 4.2815367 , 1.28237426, 1.95099611, 3.51726587,
        2.91749196]),
 array([3.51173985e+00, 2.40077568e+00, 1.49102553e-02, 2.96824333e-05,
        8.33429489e+00, 1.00137778e-01, 2.30833143e+00, 5.99968617e+00,
        2.93945814e+00, 6.70894262e+00, 1.97622765e+00, 4.14651115e+00,
        1.59928296e+00, 1.37332381e-01, 1.45432061e+00, 1.72868774e+00,
        1.15954788e+00, 3.08671126e+00, 5.99484359e+00, 1.52325123e+00,
      